In [ ]:
#due to time constraints a CNN model was taken from https://www.kaggle.com/fahadmehfoooz/brain-tumor-detection-using-keras-and-pytorch and then modified for my problem


In [ ]:
import seaborn as sns
import numpy as np 
import pandas as pd 
import cv2
import matplotlib.pyplot as plt 
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D
from tensorflow.keras.optimizers import Adam 
from kerastuner import RandomSearch
import os
import numpy as np
import pandas as pd
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
#load in images but without the images with no tumor
train_img = []
train_labels = []

test_img = []
test_labels = []

path_train = ('/kaggle/input/brain-tumor-classification-mri/Training/')
path_test = ('/kaggle/input/brain-tumor-classification-mri/Testing/')
img_size= 256

for i in os.listdir(path_train):
    for j in os.listdir(path_train+i):
        if i != 'no_tumor':
            train_img.append (cv2.resize(cv2.imread(path_train+i+'/'+j), (img_size,img_size))) 
            train_labels.append(i)
        
for i in os.listdir(path_test):
    for j in os.listdir(path_test+i):
        if i != 'no_tumor':
            test_img.append (cv2.resize(cv2.imread(path_test+i+'/'+j), (img_size,img_size))) 
            test_labels.append(i)
        
train_img = (np.array(train_img))
test_img = (np.array(test_img))


train_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(train_labels)]
test_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(test_labels)]

In [ ]:
print("Shape of train: ",(train_img).shape," and shape of test: ", (test_img).shape)

In [ ]:
#image data generator applies data augementation
img_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    )

img_datagen.fit(train_img)
img_datagen.fit(test_img)

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(np.array(train_img), np.array(train_labels), test_size = 0.1)

In [ ]:
#Build model
model = tf.keras.Sequential(
        [
          tf.keras.layers.Conv2D(kernel_size=(7,7) ,filters=64, activation='relu', padding='same'),
          tf.keras.layers.MaxPool2D(pool_size=(2,2)),
          
          tf.keras.layers.Conv2D(kernel_size=(5,5),filters=64, activation='relu', padding='same'),
          tf.keras.layers.MaxPool2D(pool_size=(2,2)),
          
          tf.keras.layers.Conv2D(kernel_size=(3,3) ,filters=64, activation='relu', padding='same'),
          tf.keras.layers.MaxPool2D(pool_size=(2,2)),
          
          tf.keras.layers.Conv2D(kernel_size=(3,3) ,filters=64, activation='relu', padding='same'),
          tf.keras.layers.MaxPool2D(pool_size=(2,2)),
          tf.keras.layers.Dropout(0.1),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dropout(0.5),
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dropout(0.5),
          tf.keras.layers.Dense(3, activation='softmax')
  ])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    


In [ ]:
history = model.fit(tf.cast(train_img, tf.float32), np.array(pd.get_dummies(train_labels)), validation_split=0.25, epochs =30, verbose=1, batch_size=64)

In [ ]:
y_pred_test = model.predict(test_img)
y_pred = []
for i in range(len(y_pred_test)):
    if y_pred_test[i].max() == y_pred_test[i][0]:
        y_pred.append(1) 
    elif y_pred_test[i].max() == y_pred_test[i][1]:
        y_pred.append(2)      
    else:
        y_pred.append(3)

In [ ]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(np.array(test_labels_encoded), np.array(y_pred))))
print('Model precision score: {0:0.4f}'. format(precision_score(np.array(test_labels_encoded), np.array(y_pred), average = "macro")))
print('Model recall score: {0:0.4f}'. format(recall_score(np.array(test_labels_encoded), np.array(y_pred), average = "macro")))
print('Model F1 score: {0:0.4f}'. format(f1_score(np.array(test_labels_encoded), np.array(y_pred), average = "macro")))

In [ ]:
print(y_pred)

In [ ]:
tf.keras.utils.plot_model(model,
                          show_shapes=True,
                          expand_nested=True,
                          to_file='model.png')